# Вычисление метрик для CUPED

**Наша цель** — научиться получать преобразованную с помощью CUPED метрику.

Для этого нужно будет написать **две функции**.

**Первая функция** вспомогательная, она будет вычислять значения целевой метрики для пользователей на произвольном промежутке времени. Первую функцию можно будет использовать, чтобы посчитать значения метрики пользователей во время пилота и до пилота.

**Вторая функция** будет вычислять непосредственно преобразованную cuped-метрику. В качестве ковариаты будем использовать значение метрики, посчитанное на периоде до начала пилота.

**Целевая метрика** — суммарная стоимость покупок пользователя за определённый период времени.

In [97]:
import numpy as np
import pandas as pd


def calculate_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name
):
    """Вычисляет значение метрики для списка пользователей в определённый период.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    period - dict, словарь с датами начала и конца периода, за который нужно посчитать метрики.
        Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}. Дата начала периода входит нужный
        полуинтервал, а дата окончание нет, то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами [user_id_name, metric_name], кол-во строк должно быть равно
        кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    
    # Фильтрую пользователей
    df_filtered = df[df[user_id_name].isin(list_user_id)]
    # Фильтрую даты
    df_filtered = df_filtered[(df_filtered[date_name] >= period['begin']) \
                              & (df_filtered[date_name] < period['end'])]
    # Сумма по пользователям
    df_g = df_filtered.groupby(user_id_name).agg(**{
        metric_name: (value_name, 'sum')
    }).reset_index()
    # Результат с кол-вом строк в list_user_id
    df_res = pd.DataFrame({user_id_name: list_user_id})
    df_res = df_res.merge(df_g, how='left', on=user_id_name).fillna(value={metric_name: 0})
    return df_res
    


def calculate_metric_cuped(
    df, value_name, user_id_name, list_user_id, date_name, periods, metric_name
):
    """Вычисляет метрики во время пилота, коварианту и преобразованную метрику cuped.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    periods - dict, словарь с датами начала и конца периода пилота и препилота.
        Пример, {
            'prepilot': {'begin': '2020-01-01', 'end': '2020-01-08'},
            'pilot': {'begin': '2020-01-08', 'end': '2020-01-15'}
        }.
        Дата начала периода входит в полуинтервал, а дата окончания нет,
        то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами
        [user_id_name, metric_name, f'{metric_name}_prepilot', f'{metric_name}_cuped'],
        кол-во строк должно быть равно кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    
    # Данные пилота
    df_pilot = calculate_metric(df,
                                   value_name,
                                   user_id_name,
                                   list_user_id,
                                   date_name,
                                   periods['pilot'],
                                   metric_name)
    # Данные предпилота
    df_prepilot = calculate_metric(df,
                                   value_name,
                                   user_id_name,
                                   list_user_id,
                                   date_name,
                                   periods['prepilot'],
                                   f'{metric_name}_prepilot')

    # Объединяю оба фрейма
    df_res = df_pilot.merge(df_prepilot, how='left', on=user_id_name)
    # cuped
    theta = np.cov(df_pilot[metric_name], df_prepilot[f'{metric_name}_prepilot'])[0, 1] \
                    / df_prepilot[f'{metric_name}_prepilot'].var()
    df_res[f'{metric_name}_cuped'] = df_res[metric_name] - theta * df_res[f'{metric_name}_prepilot']
    
    return df_res

## Проверка

In [37]:
from itertools import product

In [38]:
# Чиcло дат в пилоте
n_pilot_dates = 7

# Генерация дат в пилоте и предпилоте
dates = [str(dt)[:10] for dt in pd.date_range(start='2024-01-01', periods=2*n_pilot_dates)]
prepilot_dates = dates[:n_pilot_dates]
pilot_dates = dates[n_pilot_dates:]

# Число клиентов в датасете
n_clients = 10
control_clients = list(range(n_clients // 2))
pilot_clients = list(range(n_clients // 2, n_clients))

# Генерация датасета
df_data = pd.DataFrame(product(range(n_clients), dates), columns=['user_id', 'date'])
df_data['y'] = np.random.normal(10, 2, df_data.shape[0])

# Добавим эффект в пилоте
pilot_mask = df_data['user_id'].isin(pilot_clients) & df_data['date'].isin(pilot_dates)
df_data.loc[pilot_mask, 'y'] += np.random.normal(1, 1, pilot_mask.sum())
df_data

,user_id,date,y
0,0,2024-01-01,10.167670
1,0,2024-01-02,7.752356
2,0,2024-01-03,9.079151
3,0,2024-01-04,9.461563
4,0,2024-01-05,10.109669
...,...,...,...
135,9,2024-01-10,11.988091
136,9,2024-01-11,9.486543
137,9,2024-01-12,10.833720
138,9,2024-01-13,10.944012


In [39]:
calculate_metric(df_data,
                 'y',
                 'user_id',
                 pilot_clients,
                 'date',
                 {'begin': min(pilot_dates), 'end': max(pilot_dates)},
                 'metric')

,user_id,metric
0,5,63.826212
1,6,62.652229
2,7,74.623428
3,8,70.242890
4,9,65.489406


In [98]:
df = calculate_metric_cuped(df_data,
                       'y',
                       'user_id',
                       pilot_clients,
                       'date',
                       {
                            'prepilot': {'begin':  min(prepilot_dates), 'end': max(prepilot_dates)},
                            'pilot': {'begin': min(pilot_dates), 'end': max(pilot_dates)}
                        },
                       'metric')
df

,user_id,metric,metric_prepilot,metric_cuped
0,5,63.826212,58.338651,95.854775
1,6,62.652229,66.812170,99.332848
2,7,74.623428,60.174899,107.660111
3,8,70.242890,59.543825,102.933107
4,9,65.489406,59.448894,98.127504


In [99]:
df['metric'].var()

24.804522883571188

In [100]:
df['metric_cuped'].var()

21.339902521161534

Дисперсия снизилась